In [3]:
import gzip
import shutil
with gzip.open('wiki-Vote.txt.gz', 'rb') as f_in:
    with open('wiki-Vote.txt', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

In [4]:
lines = []
with open(r"wiki-Vote.txt", 'r') as fp:
    lines = fp.readlines()

with open(r"wiki-Vote.txt", 'w') as fp:
    for number, line in enumerate(lines):
        if number not in [0,1,2,3]:
            fp.write(line)

In [5]:
!pip install pyspark

     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 198 kB 44.6 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.0-py2.py3-none-any.whl size=281805912 sha256=ddd4b5e06d456aadb1a7f412fcb7305913f6212aa346098993255ff1304d9488
  Stored in directory: /root/.cache/pip/wheels/0b/de/d2/9be5d59d7331c6c2a7c1b6d1a4f463ce107332b1ecd4e80718
Successfully built pyspark


In [6]:
from __future__ import print_function

import re
import sys
from operator import add

from pyspark.sql import SparkSession
import os

def computeContribs(rank, urls):
    """Calculates URL contributions to the rank of other URLs."""
    for url in urls:
        yield (url, rank)


def parseNeighbors(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[0], parts[1]

def parseNeighbors_trans(urls):
    """Parses a urls pair string into urls pair."""
    parts = re.split(r'\s+', urls)
    return parts[1], parts[0]

def node_cmp(x):
  return int(x[0])


input_file = "wiki-Vote.txt"
iterations = 5
spark = SparkSession\
        .builder\
        .appName("PythonPageRank")\
        .getOrCreate()

lines = spark.read.text(input_file).rdd.map(lambda r: r[0])
all_urls = lines.map(lambda urls: parseNeighbors(urls)).distinct()
links = all_urls.groupByKey().cache()
links_transpose = lines.map(lambda urls: parseNeighbors_trans(urls)).distinct().groupByKey().cache()
result_links = links.collect()
result_transition = links_transpose.collect()
hubs = links.map(lambda url_neighbors: (url_neighbors[0], 1.0))

In [7]:
authority = None
    # Calculates and updates URL ranks continuously using PageRank algorithm.
for iteration in range(iterations):
# Calculates URL contributions to the rank of other URLs.
# calculate authority
  authority = links.join(hubs).flatMap(
    lambda x: computeContribs(x[1][1], x[1][0])).reduceByKey(add)

  max_value_a = authority.map(lambda x: x[1]).max()
  authority = authority.mapValues(lambda x: x / max_value_a)

  hubs = links_transpose.join(authority).flatMap(
    lambda x: computeContribs(x[1][1], x[1][0])).reduceByKey(add)  
  max_value_h = hubs.map(lambda x: x[1]).max()
  hubs = hubs.mapValues(lambda x: x / max_value_h)


In [8]:
result_hubs = sorted(hubs.collect(), key=node_cmp)

In [9]:
text_file = open("hub.txt", "w")
for _ in result_hubs:
  text_file.write(str(_[0]) + ",%.5f" % _[1] + "\n")
text_file.close()

result_authority = sorted(authority.collect(), key = node_cmp)
text_file = open("authority.txt", "w")
for _ in result_authority:
  text_file.write(str( _[0]) + ",%.5f" % _[1] + "\n")
text_file.close()

spark.stop()

In [17]:
def myFunc(e):
  return e[1]
result_hubs.sort(key=myFunc, reverse=True)

In [18]:
result_hubs[:10]

[('2565', 1.0),
 ('766', 0.9539738367956294),
 ('2688', 0.8109012307736835),
 ('457', 0.8083998263718667),
 ('1166', 0.7572151832677456),
 ('1549', 0.7208933608812452),
 ('11', 0.6205408521474032),
 ('1151', 0.5761852013414268),
 ('1374', 0.5628874083453793),
 ('1133', 0.493479413970747)]

In [20]:
result_authority.sort(key=myFunc, reverse=True)

In [21]:
result_authority[:10]

[('2398', 1.0),
 ('4037', 0.9977207958136023),
 ('3352', 0.9024990712420015),
 ('1549', 0.8933904532987473),
 ('762', 0.8752334923138549),
 ('1297', 0.8736172836899779),
 ('3089', 0.8734033014392443),
 ('2565', 0.8620015229598714),
 ('15', 0.8536687098722228),
 ('2625', 0.8530062804275313)]